<h1>モジュールのインポート</h1>

In [1]:
from pretreatment import pretreatment
from search_tweet_and_save import search_tweets_and_save

<h1>ライブラリのインポート</h1>

In [2]:
import pickle
from transformers import AutoTokenizer, AutoModelForSequenceClassification, LukeConfig
import torch
import numpy as np
import pandas as pd
from typing import List
import os

/home/st6322011/anaconda3/envs/NLP/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


<h1>ツイートデータの取得、ロード,前処理.ここでアカウントの情報を設定する必要があります</h1>

In [3]:
user_name=os.environ.get('AUTH_INFO_1')
mail=os.environ.get('AUTH_INFO_2')
password=os.environ.get('TWIKIT_PASSWORD')
pickle_name = "tweets"
search_tweets_and_save(query="自然言語処理 lang:ja", user_name=user_name, mail=mail, password=password, pickle_name=pickle_name, max_tweets=100, save_flag=True)

In [4]:
with open(f'{pickle_name}.pkl', 'rb') as file:
    texts = pickle.load(file)

texts_after_treat :List[str]= []
for text in texts :
    text = pretreatment(text)
    texts_after_treat.append(text)
    #print(text)
    #print("-"*100)

<h1>学習済みモデルとトークナイザーをロード</h1>

- https://huggingface.co/Mizuiro-sakura/luke-japanese-large-sentiment-analysis-wrime : で公開されていて、だれでも使うことができます
- URL先の右上の「Use in Transformers」をクリックすると、下記のコードを取得できます
- このHugging Faceでは他にも様々な学習済みモデルを利用することができます

In [5]:
tokenizer = AutoTokenizer.from_pretrained("Mizuiro-sakura/luke-japanese-large-sentiment-analysis-wrime")
model = AutoModelForSequenceClassification.from_pretrained("Mizuiro-sakura/luke-japanese-large-sentiment-analysis-wrime")

/home/st6322011/anaconda3/envs/NLP/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


<h1>ツイートデータを感情分析する前処理</h1>

In [6]:
col_names = ['喜び', '悲しみ', '期待', '驚き', '怒り', '恐れ', '嫌悪', '信頼','text']

In [7]:
df = pd.DataFrame(columns=col_names)

In [8]:
df

,喜び,悲しみ,期待,驚き,怒り,恐れ,嫌悪,信頼,text


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 0 entries
Data columns (total 9 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   喜び      0 non-null      object
 1   悲しみ     0 non-null      object
 2   期待      0 non-null      object
 3   驚き      0 non-null      object
 4   怒り      0 non-null      object
 5   恐れ      0 non-null      object
 6   嫌悪      0 non-null      object
 7   信頼      0 non-null      object
 8   text    0 non-null      object
dtypes: object(9)
memory usage: 132.0+ bytes


In [10]:
#この時点で、八個の感情を表す列と、一個のテキストを表す列が作成できた。
# 'text'列を除くすべての列をfloatに変換
float_columns = col_names[:-1]  # 最後の'text'列を除くすべての列名
df[float_columns] = df[float_columns].astype(float)

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 0 entries
Data columns (total 9 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   喜び      0 non-null      float64
 1   悲しみ     0 non-null      float64
 2   期待      0 non-null      float64
 3   驚き      0 non-null      float64
 4   怒り      0 non-null      float64
 5   恐れ      0 non-null      float64
 6   嫌悪      0 non-null      float64
 7   信頼      0 non-null      float64
 8   text    0 non-null      object 
dtypes: float64(8), object(1)
memory usage: 132.0+ bytes


<h1>ツイートを感情分析</h1>

In [12]:
batch_size = 4
# バッチ処理のためにデータを分割
batched_inputs :List[ List[str] ]= [texts_after_treat[i:i + batch_size] for i in range(0, len(texts_after_treat), batch_size)]
"""
range(0, len(texts_after_treat), batch_size) : batch_sizeの間隔で 0 から len(texts_after_treat)までデータを取り出す
texts_after_treat[i:i + batch_size] : i番目からi + batch_sizeまでのテキストのリスト
"""

max_seq_length = 512

In [13]:
# バッチごとに処理
for batch in batched_inputs:
    # バッチ内の全テキストを一度にトークナイズ
    encoded_input = tokenizer(batch, padding=True, truncation=True, max_length=max_seq_length, return_tensors="pt")
    input_ids :torch.Tensor= encoded_input['input_ids']
    attention_mask :torch.Tensor= encoded_input['attention_mask']
    
    # モデルに入力
    with torch.no_grad():
        outputs = model(input_ids, attention_mask=attention_mask)
    
    # ロジットを取得し、numpy配列に変換
    logits :np.ndarray= outputs.logits.detach().numpy()

    # バッチ内の各テキストに対する処理
    for i, output in enumerate(logits):
        tweet :str= batch[i]
        output :np.ndarray= output.astype(np.float64)
        new_row :List[float | str]= list(output) + [tweet]
        df.loc[len(df)] = new_row

In [14]:
df.to_pickle('saved_dataframe.pkl')